In [1]:
import random, numpy as np, pandas as pd
from matplotlib import pyplot as plt
alphap = 2
probp = 0.75
expon = 2 * alphap - 1
slope = 0.77872

In [2]:
def betadist(alpha):
    """gives a random number from beta distribution"""
    return random.betavariate(alpha,alpha)

In [3]:
def decision(probability):
    """
    decides with a given probability whether to keep the right part
    """
    if float(probability) > random.random():
        return True
    else: 
        return False

In [4]:
def splitting(segment):
    """
    splits a given segment. left and right are endpoints of the segment
    returns : 
        xL -> length of the left segment
        xR -> length of the right segment
        flag -> keeping the right segment
        xLp, xRp -> probability(unnormalized) for being selected
        change -> change of normalization const
    """
    xL = segment * betadist(alphap)
    xR = segment - xL
    flag = decision(probp)
    xLp = xL**expon
    xRp = xR**expon
    change = xLp + xRp - segment**expon
    return xL, xR, flag, xLp, xRp, change

In [5]:
# def pickindex(frag_prob, frag_prob_sum):
#     """
#     picks up a segment to be subsequently split
#     """
#     r = random.uniform(0, frag_prob_sum)
#     sum_ = 0
#     for index in range(len(frag_prob)):
#         sum_ += frag_prob[index]
#         if sum_ < r:
#             continue
#         else:
#             return index

In [6]:
def pickindex(frag_prob, frac_sum):
    """
    picks up a segment to be subsequently split
    """
    r = random.uniform(0,1)
    
    if r <= frac_sum:
        sum_ = 0
        for index in range(len(frag_prob)):
            sum_ += frag_prob[index]
            if sum_ < r:
                continue
            else:
                return index

In [7]:
def number_length(segment_lengths,flags):
    
    N = 0
    M = 0

    for i in range(len(flags)):
        if flags[i]:
            N += 1
            M += segment_lengths[i]
            pass
        pass
    return N, M

In [8]:
fractal_dim = slope

def fractal_length(segment_lengths,flags):

    M_frac = 0

    for i in range(len(flags)):
        if flags[i]:
            M_frac += segment_lengths[i]**fractal_dim
            pass
        pass
    return M_frac

In [9]:
def realization_value(total_iteration, min_iteration, iteration_step, data_point = False):
    
    lengths = [1.]
    flags = [True]
    frag_prob = [1.] # raw probability, not normalized
    #frag_prob_sum = 1.0 # normalization const
    frac_sum = 1.0

    iteration_list = list(range(min_iteration, total_iteration + 1, iteration_step))
    
    M_realization = []
    
    for i in range(total_iteration + 1):
        
        #index = pickindex(frag_prob, frag_prob_sum)
        index = pickindex(frag_prob, frac_sum)
        
        if index != None and flags[index] == True:

            xL, xR, flag, xLp, xRp, change = splitting(lengths[index])
            
            lengths[index] = xL
            lengths.append(xR)
            flags.append(flag)
            frag_prob[index] = xLp 
            frag_prob.append(xRp)
            #frag_prob_sum += change
            frac_sum += change
            pass
        
        if i+1 in iteration_list:
            M_frac = fractal_length(lengths,flags)
            M_realization.append(M_frac)
        pass 
      
    if data_point:
        print(np.array(M_realization))
    
    else:
        M_frac = fractal_length(lengths,flags)
        return M_frac

In [10]:
def ensemble_average(total_iteration, ensemble_size):

    M_ensemble = 0
    
    for i in range(ensemble_size):
        M_list = realization_value(total_iteration, total_iteration, total_iteration)
        M_ensemble += M_list
        pass
    
    M_average = M_ensemble/ensemble_size
    
    print(M_average)

In [11]:
for i in range(5):
    realization_value(1_000_000, 100_000, 100_000, data_point = True)

[1.24806469 1.24745653 1.2768754  1.28457667 1.26802612 1.25909904
 1.26889705 1.29310097 1.29961146 1.26903696]
[0.63747689 0.59162685 0.58078905 0.586857   0.57327186 0.57327186
 0.56431903 0.56081803 0.55816033 0.56569273]
[0.81603    0.8330162  0.85548976 0.77350068 0.77913772 0.79550429
 0.79745072 0.80201727 0.80201727 0.80655571]
[1.01437566 1.0626362  1.03968601 1.00424786 0.98168311 0.95825409
 0.94994785 0.91759261 0.92251168 0.90655313]
[0.81962479 0.85775311 0.86136179 0.87818274 0.85989647 0.85989647
 0.84098014 0.82924976 0.83650882 0.83650882]


In [12]:
for i in range(10):
    a_ = 1_000*(i+1)
    ensemble_average(a_, 100_000)

1.002222350446228
0.9994647460374152
1.0017092216944123
1.0006375106911432
0.9997697727008863
1.0032827311824999
1.0016352174940713
0.9993603056383773
1.0000629785441975
1.0006113165299664
